[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GP211/2023-fall-class-notebooks/blob/main/in-class/004-vesuvio.ipynb)

# Vesuvius phase unwrapping

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
!python3 -m pip install "giee @ git+https://github.com/GP211/2023-fall-class-notebooks.git@d00084220a6501d5fe744869f82e64b3dab9c03b" 

In [ ]:
import sep_python
io=sep_python.default_io
vec=io.vector_from_storage("../data/vesuvio.H")

In [ ]:
x=vec.get_nd_array()
print(type(x),x.dtype,vec.get_data_type())

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('bokeh','matplotlib')

vec._hyper.axes=vec._hyper.axes[0:2]

amp=io.get_reg_vector(np.absolute(vec.get_nd_array()),hyper=vec.get_hyper())
phase=io.get_reg_vector(np.angle(vec.get_nd_array()),hyper=vec.get_hyper())

print(amp.get_data_type(),vec.min(),vec.max())
import sep_plot
sep_plot.Grey(amp,bpclip=0,epclip=96)+sep_plot.Grey(phase)


These images are made from
backscatter signals s1(t) and s2(t), recorded along two satellite orbits 800-km high and
54-m apart. The signals are very high frequency (the radar wavelength being 2.7 cm).
The signals were Fourier transformed and one multiplied by the complex conjugate of the
other, getting the product $Z = S_1(ω)S_2(ω)$. The product’s amplitude and phase are shown
in Figure 2.8. Examining the data, you can notice that where the signals are strongest
(darkest on the left), the phase (on the right) is the most spatially consistent.

In the figure in the left we see that contours of constant phase
appear to be contours of constant altitude;
this conclusion leads us to suppose that a study of radar theory
would lead us to a relation like $Z(x,y)=e^{ih(x,y)}$,
where $h(x,y)$ is altitude.
We nonradar specialists often think of phase in
$e^{i\phi} = e^{i\omega t_0(x,y)}$
as being caused by some time delay and
being defined for some constant frequency $\omega$.
Knowledge of this $\omega$ (as well as some angle parameters)
would define the physical units of $h(x,y)$.

Because the flat land away from the mountain is all at the same phase
(as is the altitude),
the distance as revealed by the phase does not represent
the distance from the ground to the satellite viewer.
We are accustomed to measuring altitude along a vertical line to a datum;
but here, the distance seems to be measured
from the ground along a $23^\circ$ angle from the vertical
to a datum at the satellite height.

Phase is a troublesome measurement,
because we generally see it modulo $2\pi$.
Marching up the mountain, we see the phase getting lighter and lighter
until it suddenly jumps to black,
which then continues to lighten
as we continue up the mountain to the next jump.
Let us undertake to compute the phase,
including all its jumps of $2\pi$.
Begin with a complex number $Z$ representing
the complex-valued image at any location
in the $(x,y)$-plane.

\begin{eqnarray}
r e^{i \phi}   &=& Z \\
\ln |r| + i \phi &=& \ln Z \\
\phi(x,y)            &=&  \Im \ln Z(x,y) ~+~  2\pi N(x,y)
\end{eqnarray}
Computers find the imaginary part of the logarithm
with the arctan function of two arguments, atan2(y,x),
which puts the phase in the range $-\pi < \phi \le \pi$,
although any multiple of $2\pi$ could be added.
We seem to escape the $2\pi N$ phase ambiguity by differentiating:
\begin{eqnarray}
{\partial\phi \over \partial x} =\Im {1 \over Z}{\partial Z \over \partial x} =
                                   {\Im  \bar Z {\partial Z \over \partial x} \over \bar Z Z }
\end{eqnarray}

For every point on the $y$-axis, The above equation
is a differential equation on the $x$-axis.
We could integrate them all to find $\phi(x,y)$.
That sounds easy.
On the other hand,
the same equations are valid when $x$ and $y$ are interchanged,
therefore we get twice as many equations as unknowns.
Ideally either of these sets of equations
is equivalent to the other;
but for real data, we expect to be fitting this fitting goal:

\begin{equation}
\nabla \phi \quad \approx \quad {\Im  \bar Z \nabla Z \over \bar Z Z}
\end{equation}
where
$\nabla = ({\partial \over \partial x}, {\partial \over \partial y} ) $.
Mathematically, computing phase this way
is like our previous seismic flattening with
$\nabla \tau \approx {\bf d}$.
Taking measurements to be phase differences
between neighboring mesh points,
it is more correct to interpret the above as
a difference equation than a differential equation.
Because we measure phase differences only over tiny distances (one pixel),
we hope not to worry about phases greater than $2\pi$.
But, if such jumps do occur, the jumps contribute to overall error.
\par
Let us consider a typical location in the $(x , y )$ plane where the complex numbers
$Z_{i,j}$ are given. Define a shorthand $a , b, c$, and $d$ as follows:


\begin{equation}
        \left[
                \begin{array}{ll}
                a & b \\
                c & d
                \end{array}
        \right]
        =
        \left[
                \begin{array}{ll}
                Z_{i,j}   & Z_{i,j+1} \\
                Z_{i+1,j} & Z_{i+1,j+1}
                \end{array}
        \right]
\end{equation}
With this shorthand, the difference equation representation of the fitting goal is:
\begin{equation}
        \begin{array}{rcl}
                \phi_{i+1,j} -\phi_{i,j} &\approx & \Delta\phi_{ac} \\
                \phi_{i,j+1} -\phi_{i,j} &\approx & \Delta\phi_{ab}
        \end{array}
\end{equation}
Now,
let us find the phase jumps between the various locations. Complex numbers $a$ and $b$ may be expressed in polar form, say $a=r_ae^{i\phi_a}$ and $b=r_be^{i\phi_b}$.
The complex number
$\bar a b = r_a r_b e^{i(\phi_b-\phi_a)}$ has the desired phase
$\Delta \phi_{ab}$.

To obtain it we take the imaginary part of the complex logarithm
$\ln |r_a r_b| + i\Delta \phi_{ab}$:
\begin{equation}
  \begin{array}{lllll}
        \phi_b-\phi_a &=& \Delta \phi_{ab} &=& \Im \ln  \bar a b\\
        \phi_d-\phi_c &=& \Delta \phi_{cd} &=& \Im \ln  \bar c d\\
        \phi_c-\phi_a &=& \Delta \phi_{ac} &=& \Im \ln  \bar a c\\
        \phi_d-\phi_b &=& \Delta \phi_{bd} &=& \Im \ln  \bar b d
  \end{array}
\end{equation}
which gives the information needed to fill in the right side.

The operator needed is
gradient with its adjoint, the divergence.


In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
real=io.get_reg_vector(np.real(vec.get_nd_array()),hyper=vec.get_hyper())
imag=io.get_reg_vector(np.imag(vec.get_nd_array()),hyper=vec.get_hyper())

print(real.min(),real.max(),imag.min(),imag.max())
import sep_plot
sep_plot.Grey(real,bpclip=0,epclip=100)+sep_plot.Grey(imag)

In [ ]:
from sep_python import Hypercube
def make_data(cdata):
  ns=cdata.get_hyper().get_ns()
  ns.append(2)
  hyper=Hypercube.set_with_ns(ns)
  dat=io.get_reg_vector(hyper,data_format=np.float32)
  dat.zero()
  cin=cdata.get_nd_array()
  d=dat.get_nd_array()
  print(d.shape,hyper.get_ns())

  a=cin[1:-1,1:-1]
  c=cin[1:-1,2:]
  b=cin[2:,1:-1]

  d[0,1:-1,1:-1]=np.imag(np.log(c*np.conjugate(a)))
  d[1,1:-1,1:-1]=np.imag(np.log(b*np.conjugate(a)))
  return dat


data=make_data(vec)


In [ ]:
from generic_solver import ProblemL2Linear
from giee import Igrad2
print(data.min(),data.max())

model=io.get_reg_vector(vec.get_hyper(),dataFromat="float32")
lop=Igrad2(model,data)
lop.dotTest(verbose=True)



In [ ]:
x=model.clone()
print(data.min(),data.max())

In [ ]:
print(model.arr.max())

In [ ]:
%xmode verbose
from generic_solver import LCGsolver
from generic_solver import BasicStopper 
model.zero()
problem=ProblemL2Linear(model,data,lop)
stopper=BasicStopper(niter=500)
solver=LCGsolver(stopper)
solver.setDefaults(save_obj=True,save_res=True,iter_sampling=5)
problem.data.dot(problem.data)
solver.run(problem,verbose=True)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.res,bpclip=5,epclip=95)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.model,bpclip=0,epclip=100)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(solver.obj[0:])

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(solver.res[0],bpclip=5,epclip=95)

In [ ]:
import scipy.signal
import holoviews as hv
hv.extension('bokeh','matplotlib')
real_new=real.clone()
imag_new=imag.clone()
real_new.get_nd_array()[:]=scipy.signal.medfilt2d(real.get_nd_array(), kernel_size=7)
imag_new.get_nd_array()[:]=scipy.signal.medfilt2d(imag.get_nd_array(), kernel_size=7)

sep_plot.Grey(real_new,bpclip=0,epclip=100)+sep_plot.Grey(imag_new)

In [ ]:

import holoviews as hv
hv.extension('bokeh','matplotlib')
complexD=real_new.get_nd_array()+imag_new.get_nd_array()*1.j
vec_new=io.get_reg_vector(complexD,hyper=vec.get_hyper())
amp_new=io.get_reg_vector(np.absolute(complexD),hyper=vec.get_hyper())
phase_new=io.get_reg_vector(np.angle(complexD),hyper=vec.get_hyper())
phaseD=phase_new.clone()
phaseD.scale_add(phase,sc2=-1.)
sep_plot.Grey(amp_new,bpclip=3,epclip=97)+sep_plot.Grey(phase_new)


In [ ]:
data_new=make_data(vec_new)
model.zero()
problem=ProblemL2Linear(model,data_new,lop)
stopper=BasicStopper(niter=500)
solver=LCGsolver(stopper)
solver.setDefaults(save_obj=True,save_res=True,iter_sampling=5)
problem.data.dot(problem.data)
solver.run(problem,verbose=True)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(solver.obj[0:])

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.res,bpclip=0,epclip=100)

In [ ]:
import holoviews as hv
hv.extension('bokeh','matplotlib')
sep_plot.Grey(problem.model,bpclip=0,epclip=100)